# Deploy de um Projeto de Machine Learnin com Flask: Previsão de Falha de Maquinário

Quando estou sobre dados, principalmente voltados a aprendizado de máquina, nos deparamos com uma imensidade de projetos sobre o assunto mostrando uma variedade de métricas, algoritmos, tipos de modelos, métodos de ajustes de parâmetros e muitos outros aspecto de construção de um algoritmo de aprendizagem de máquina. 

Mas o que acontece depois de achar o modelo ideal? O que acontece após encontrar com 99,99% de acurácia um modelo que prevê o cancelamento de um serviço? O faturamento do próximo mês?

Acredito que boa parte dos projetos de Machine Learning param nesta etapa, ou porque o objetivo era esse mesmo ou por falta de prósito final para o algoritmo. Pensando nisso, vamos nesse post desenvolver um modelo de machine learning e fazer o deploy desse modelo por meio de uma API com a biblioteca Flask do python 

## Sobre o Projeto
### Entedimento do Problema

Este conjunto de dados contém dados de sensores coletados de várias máquinas, com o objetivo de prever falhas de máquinas com antecedência. Ele inclui uma variedade de leituras de sensores, bem como as falhas de máquinas registradas.

Segue o nome a descrição das colunas:

footfall: O número de pessoas ou objetos passando pela máquina.

tempMode: O modo de temperatura ou configuração da máquin



AQ: Índice de qualidade do ar perto da máquina.

USS: Dados do sensor ultrassônico, indicando medições de proximidade.

CS: Leituras atuais do sensor, indicando o uso de corrente elétrica da máquina.

VOC: Nível de compostos orgânicos voláteis detectado perto da máquina.

RP: Posição rotacional ou RPM (revoluções por minuto) das peças da máquina.

IP: Pressão de entrada na máquina.

Temperature: A temperatura operacional da máquina.

fail: Indicador binário de falha da máquina (1 para falha, 0 para nenhuma falha).ra nenhuma falha).a falha).

## Importação e Análises Iniciais

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('archive/data.csv')

In [5]:
df.tail()

,footfall,tempMode,AQ,USS,CS,VOC,RP,IP,Temperature,fail
939,0,7,7,1,6,4,73,6,24,1
940,0,7,5,2,6,6,50,6,24,1
941,0,3,6,2,7,5,43,6,24,1
942,0,6,6,2,5,6,46,7,24,1
943,18,7,4,2,6,3,61,7,24,1


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 944 entries, 0 to 943
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   footfall     944 non-null    int64
 1   tempMode     944 non-null    int64
 2   AQ           944 non-null    int64
 3   USS          944 non-null    int64
 4   CS           944 non-null    int64
 5   VOC          944 non-null    int64
 6   RP           944 non-null    int64
 7   IP           944 non-null    int64
 8   Temperature  944 non-null    int64
 9   fail         944 non-null    int64
dtypes: int64(10)
memory usage: 73.9 KB


In [45]:
#Verificando se a base de dados esta desbalanceada
df['fail'].value_counts(normalize=True)*100

fail
0    58.368644
1    41.631356
Name: proportion, dtype: float64

Sobre a base dados, ela possui apeas 994 registros, a nossa variável alvo é a coluna 'fail', para a construção do modelo precisa ser aplicado algum método de normalização e a base de dados está um pouco desbalanceada com o valor da variável alvo com 41,6% dos registros

### Treinamento e Avaliação do Modelo
Para o treinamento desse modelo irei utilizar o pycaret, se precisar de mais ferramentas sobre essa biblioteca sugiro visitar o post nesse link.

In [46]:
from pycaret import classification
from pycaret.classification import *

In [47]:
inicialização = setup(df, target='fail', normalize=True, fix_imbalance=True)

,Description,Value
0,Session id,6313
1,Target,fail
2,Target type,Binary
3,Original data shape,"(944, 10)"
4,Transformed data shape,"(1054, 10)"
5,Transformed train set shape,"(770, 10)"
6,Transformed test set shape,"(284, 10)"
7,Numeric features,9
8,Preprocess,True
9,Imputation type,simple


Não vou adentrar muito sobre a questão da construção do modelo, esse não é o objetivo desse post, caso prefira pode acessar a própria documentação do pycaret que é simples e completa.

Agora vamos verificar qual o melhor modelo para esse projeto pelo comando 'compare_models()' 

In [48]:
best_model = compare_models(include =['rf', 'gbc', 'svm', 'ridge'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.9106,0.9654,0.9308,0.8702,0.8980,0.8190,0.8226,0.0110
rf,Random Forest Classifier,0.9000,0.9704,0.8836,0.8839,0.8816,0.7953,0.7980,0.0530
gbc,Gradient Boosting Classifier,0.8955,0.9699,0.8799,0.8768,0.8763,0.7859,0.7887,0.0330
svm,SVM - Linear Kernel,0.8894,0.9430,0.8729,0.8692,0.8684,0.7733,0.7767,0.0100


Para a esolha dos modelos de machine learning passei quatro opções, o Gradient Boosting, Ridge, Random Forest e o SVM.
O pycaret identificou que o modelo que mais se adequa à conjuntura do projeto é o Ridge , e agora vamos criar esse modelo e passar para a etapa de testes do modelo

In [49]:
model = create_model(best_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9545,0.9801,0.9630,0.9286,0.9455,0.9065,0.9070
1,0.8030,0.9259,0.8519,0.7188,0.7797,0.6039,0.6110
2,0.9697,0.9810,0.9630,0.9630,0.9630,0.9373,0.9373
3,0.9091,0.9668,0.8889,0.8889,0.8889,0.8120,0.8120
4,0.8485,0.9639,0.9630,0.7429,0.8387,0.7003,0.7213
5,0.9242,0.9615,0.9286,0.8966,0.9123,0.8457,0.8461
6,0.9091,0.9389,0.9286,0.8667,0.8966,0.8156,0.8172
7,0.9242,0.9878,0.8929,0.9259,0.9091,0.8442,0.8446
8,0.9242,0.9690,0.9643,0.8710,0.9153,0.8471,0.8507


### Testes e Finalização da Etapa de Modelagem 

In [50]:
evaluate_model(model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

Com o comando 'evaluate_model()' conseguimos observar de forma dinâmica uma série de métricas sobre o modelo. Um ponto que chamou a atenção foi na análise de 'Feature Importance' que nos dá a visão do percentual de importância que cada variável dentro do modelo, a importância da coluna VOC na variação da variável alvo.

Agora vamos finalizar o modelo passando toda a base de dados para treino e salvando o modelo de previsão em um arquivo para ser utilizado no deploy.

In [56]:
final_best = finalize_model(model)
save_model(final_best, 'Prev_Maq')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['footfall', 'tempMode', 'AQ',
                                              'USS', 'CS', 'VOC', 'RP', 'IP',
                                              'Temperature'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strategy='mean'))),
                 ('categorical_imputer',
                  TransformerWrap...
                                                                               ran

## Considerações Finais sobre a etapa de modelagem

O objetivo até aqui foi chegar a um modelo de machine learning sem se aprofundar muito em cada etapa, o modelo pode ser melhorado e ele deve ser melhorado quando utilizado para produção, para fins didáticos como este post o resultado encontrado é o suficiente para avançarmos para a próxima etapa.

# Criação da API para Deploy do Modelo

Agora que já temos o modelo salvo co o nome 'Prev_Maq', vamos precisar do VSCode para construir a API com o Flask e do Postman para realizar a requisição das previsões.